In [1]:
#Import all reuired libraries
import pandas as pd
import datetime
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
from datetime import timedelta
from sklearn import preprocessing
import category_encoders as ce 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import os


PROJ: proj_create_from_database: Cannot find proj.db


The primary dataset which will be used in this analysis is the victim based part 1 crime table, produced by the Baltimore City Police Department. It reports crimes based upon each victim, rather than occurance, and thus each row can be thought of as a victim. 

In [2]:
#Crime = pd.read_csv('https://data.baltimorecity.gov/api/views/wsfq-mvij/rows.csv?accessType=DOWNLOAD')
Crime = pd.read_csv('C:/Users/cyeager/Desktop/UMBC/DATA 602/Homework2/Crime.csv',sep=",")

In [3]:
print(Crime.columns)

Index(['Unnamed: 0', 'CrimeDate', 'CrimeTime', 'CrimeCode', 'Location',
       'Description', 'Inside/Outside', 'Weapon', 'Post', 'District',
       'Neighborhood', 'Longitude', 'Latitude', 'Location 1', 'Premise',
       'vri_name1', 'Total Incidents'],
      dtype='object')


In [5]:
#Keep only the columns we want
df=Crime.filter(['CrimeDate', 'CrimeTime','Description','Latitude','Longitude'])

In [6]:
#Concat the date and time columns into a single feature
df['DT']=df['CrimeDate'] + ' ' + df['CrimeTime']
#df

,CrimeDate,CrimeTime,Description,Latitude,Longitude,DT
0,10/31/2020,10:57:00,ROBBERY - STREET,39.288013,-76.651406,10/31/2020 10:57:00
1,10/31/2020,18:19:00,COMMON ASSAULT,39.293719,-76.595851,10/31/2020 18:19:00
2,10/31/2020,14:00:00,LARCENY,39.353147,-76.588293,10/31/2020 14:00:00
3,10/31/2020,17:20:00,SHOOTING,39.322920,-76.594384,10/31/2020 17:20:00
4,10/31/2020,00:43:40,HOMICIDE,39.361468,-76.709190,10/31/2020 00:43:40
...,...,...,...,...,...,...
319456,05/01/1977,00:01:00,RAPE,39.328817,-76.626880,05/01/1977 00:01:00
319457,06/01/1975,00:00:00,RAPE,39.289582,-76.691297,06/01/1975 00:00:00
319458,07/01/1973,23:00:00,RAPE,39.326183,-76.687168,07/01/1973 23:00:00
319459,07/20/1969,21:00:00,RAPE,39.358898,-76.635300,07/20/1969 21:00:00


In [7]:
#reformat into a new datetime column
df['DT']=pd.to_datetime(df['DT'])
#df

,CrimeDate,CrimeTime,Description,Latitude,Longitude,DT
0,10/31/2020,10:57:00,ROBBERY - STREET,39.288013,-76.651406,2020-10-31 10:57:00
1,10/31/2020,18:19:00,COMMON ASSAULT,39.293719,-76.595851,2020-10-31 18:19:00
2,10/31/2020,14:00:00,LARCENY,39.353147,-76.588293,2020-10-31 14:00:00
3,10/31/2020,17:20:00,SHOOTING,39.322920,-76.594384,2020-10-31 17:20:00
4,10/31/2020,00:43:40,HOMICIDE,39.361468,-76.709190,2020-10-31 00:43:40
...,...,...,...,...,...,...
319456,05/01/1977,00:01:00,RAPE,39.328817,-76.626880,1977-05-01 00:01:00
319457,06/01/1975,00:00:00,RAPE,39.289582,-76.691297,1975-06-01 00:00:00
319458,07/01/1973,23:00:00,RAPE,39.326183,-76.687168,1973-07-01 23:00:00
319459,07/20/1969,21:00:00,RAPE,39.358898,-76.635300,1969-07-20 21:00:00


Here we zip the latitude and longitude features into one, then apply the point function so that they can be displayed as a single dot of a map for each crime.

In [8]:
df['Coordinates'] = list(zip(df['Longitude'], df['Latitude']))
df['Coordinates'] = df['Coordinates'].apply(Point)

In [9]:
#All data prior to Jan 1 2014 was found to be sparse and inconsistent and thus is removed
df=df[(df['DT'] >= '2014-01-01 00:00:01')]
df['DT'].min()

Timestamp('2014-01-01 00:01:00')

In [10]:
#Save of a copy of this to come back to if we mess it up
df_saved=df

In [11]:
#Save of a copy of this to come back to if we mess it up
df_saved.to_pickle('CrimemapData.pkl')

In [ ]:
df = pd.read_pickle('CrimemapData.pkl')

In [12]:
#Remove na values, only a few are lost in proportion to overall size, not worth investigating to potentially fix or interpolate
print(df.shape)
df=df.dropna(axis=0,how='any')
df=df.dropna(axis=1,how='any')
print(df.shape)

(319337, 7)
(319026, 7)


We are going to break this data down into 2 hours intervals of time where we look at all of the crime which is taking place in the city during that snapshot and see if we can find a pattern that acts as a precursor to homicides/shootings. The first thing we do here is figure out how many 2 hours groups we can get from this dataset:

In [13]:
print(df['DT'].min())
print(df['DT'].max())
T=(df['DT'].max())-(df['DT'].min())

2014-01-01 00:01:00
2020-10-31 22:20:00


In [47]:
#here we find the total number of hours contained within our df and then see how many 2 hour chunks there are within in.
#We can create 29951 images 
days, seconds = T.days, T.seconds
hours = days * 24 + seconds // 3600
hours/2

29951.0

In [15]:
#We want each type of crime to be a specific color
ClusterColors = ['#E47A2E', '#3F69AA', '#D5AE41', '#BD3D3A','#BD3D3A',"#006E6D",'#7F4145','#223A5E','#BD3D3A', '#3F69AA', '#D5AE41', '#766F57','#E47A2E',"#006E6D",'#7F4145','#223A5E',"#9C9A40","#9C9A40",'blue','green','red','purple','brown','#BD3D3A', '#3F69AA', '#D5AE41', '#766F57','#E47A2E',"#006E6D",'#7F4145','#223A5E',"#9C9A40"]

In [17]:
#Here we assign the color and size to each crime type point
CrimeColours = list(zip(list(df['Description'].unique()), ClusterColors))
CrimeColours=pd.DataFrame(CrimeColours)
CrimeColours['shape']=50

In [20]:
#Rename Colmns
CrimeColours.columns=['crime','color','shape']

In [21]:
df = pd.merge(df, CrimeColours, left_on=["Description"], right_on=['crime'])

Running the loop to create almost 30k maps is very time consuming. Via trial and error it was found that the df had a few flaws which were not immediately noticeable. One of these was that a hadful of points actually fell outside the city due to geocoding errors. Here we find those points via spatially joining all of the data to a polygon of the city, then filtering the main df of those without a match.

In [ ]:
Outsidezbmore=gpd.sjoin(BaltimoreCommunities,gpd.GeoDataFrame(df, geometry='Coordinates').set_crs("EPSG:4326"), op='contains')

In [ ]:
Outsidezbmore.index_right
main_list = np.setdiff1d(list(df.index),list(Outsidezbmore.index_right))
main_list

In [ ]:
print(df.shape)
df=df.drop(df.index[main_list])
print(df.shape)

In [ ]:
#save it again
df.to_pickle('CrimemapDataClensed.pkl')

In [2]:
df = pd.read_pickle('CrimemapDataClensed.pkl')
df.head()

,CrimeDate,CrimeTime,Description,Latitude,Longitude,DT,Coordinates,crime,color,shape
0,10/31/2020,10:57:00,ROBBERY - STREET,39.288013,-76.651406,2020-10-31 10:57:00,POINT (-76.65141 39.28801),ROBBERY - STREET,#E47A2E,50
1,10/31/2020,16:18:00,ROBBERY - STREET,39.248946,-76.639765,2020-10-31 16:18:00,POINT (-76.63977 39.24895),ROBBERY - STREET,#E47A2E,50
2,10/31/2020,10:57:00,ROBBERY - STREET,39.288013,-76.651406,2020-10-31 10:57:00,POINT (-76.65141 39.28801),ROBBERY - STREET,#E47A2E,50
3,10/31/2020,10:45:00,ROBBERY - STREET,39.309795,-76.648154,2020-10-31 10:45:00,POINT (-76.64815 39.30980),ROBBERY - STREET,#E47A2E,50
4,10/30/2020,14:43:00,ROBBERY - STREET,39.352462,-76.595359,2020-10-30 14:43:00,POINT (-76.59536 39.35246),ROBBERY - STREET,#E47A2E,50


Finally, the loop below creates a new df for each 2 hour window within the master dataframe, then appends each of these new, smaller dfs to a list.

In [4]:
maybe=[]
X=df['DT'].min()
Y=df['DT'].min()- timedelta(hours=2)
for i in range(29950):
    X=X + timedelta(hours=2)
    Y=Y + timedelta(hours=2)
    maybe.append(df[(df.DT >=Y) & (df.DT <=X)])
        
    #print(str(Y)+" - "+str(X))

We create a list of the 2 hour dfs which contain either homicide or shooting for identification purposes since this is a classification model. Using this data we will place each image into a folder that fits its classification (homicide/shooting or none).

In [5]:
searchfor=["SHOOTING","HOMICIDE"]
um=[]
for i in range(len(maybe)):
    
    um.append(maybe[i][maybe[i]['Description'].str.contains('|'.join(searchfor))].empty)

In [8]:
#Read in the neighborhoods boundary shapefile from API
BaltimoreCommunities = gpd.read_file('https://data.baltimorecity.gov/api/geospatial/peh2-3qyi?method=export&format=Shapefile')

In [9]:
len(maybe)

29950

The final data issue to overcome was filtering those 2 hour windows in which no crime at all took place. These would cause an error as the mapping operation is disrupted by empty dfs. Rather than deleted them, I simply incorporated an if statement with 'continue' line into the main mapping loop.

In [10]:
dud=[]
for i in range(len(maybe)):
    dud.append(maybe[i].empty)

In [11]:
skiplist=[i for i,x in enumerate(dud) if x == True]
#skiplist

In [13]:
#Only 137 of the nearly 30k 2 hour windows contained no crime.
len(skiplist)

137

This loop creates one map image for each 2 hour window. A point is displayed for each crime, it has a specific color which corresponds to its type. I've also spatially joined crime to the neighborhood in which it belongs and created a red outline for each neighbrhood where there was crime, all others are outlined in black. Once the map is created its given a unique name and saved as a .png file into a folder which dsignates its status as having a homicide or not. 

In [ ]:
for i in range(0,len(maybe)):
    if maybe[i].empty==True:
        continue    
    base = BaltimoreCommunities.plot(color='none', edgecolor='black',alpha=.5)

    base.set_aspect('equal')
    base.set_axis_off()

    ax=gpd.GeoDataFrame(maybe[i], geometry='Coordinates').set_crs("EPSG:4326").plot(ax=base,c=maybe[i]['color'], markersize = maybe[i]['shape'],alpha=.7)

    gpd.sjoin(BaltimoreCommunities,gpd.GeoDataFrame(maybe[i], geometry='Coordinates').set_crs("EPSG:4326"), op='contains').plot(ax=base,edgecolor='red',color='none')
  
    fig = ax.figure


    File=(os.getcwd()+"\Maps"+"\\"+str(um[i])+"\\"+maybe[i].DT.max().strftime(format='%Y%m_%I%M')+str(i)+".png")
    fig.savefig(File, bbox_inches = 'tight',pad_inches = 0)
    plt.close('all')

Out of our 29951 images, 'False' designates that there was a homicide or shooting during that 2 hour window. There were 4824 of these. The remaining 24989 (designated 'True') were windows of time without a shooting or homicide. 